In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local").appName("spart-dataframe").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/28 14:33:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
filepath = "/home/ubuntu/working/spark/data/titanic_train.csv"

# inferSchema : 컬럼 타입을 자동으로 추론
titanic_sdf = spark.read.csv(filepath, inferSchema=True, header=True)

In [6]:
import pandas as pd

titanic_pdf = pd.read_csv(filepath)

In [8]:
# spark 에서 order by 사용
import pyspark.sql.functions as F

titanic_sdf.orderBy(
    F.col("Pclass"),
    ascending=False
).show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|         25|       0|     3|Palsson, Miss. To...|female| 8.0|    3|    1|          349909| 21.075| null|       S|
|         47|       0|     3|   Lennon, Mr. Denis|  male|null|    1|    0|          370371|   15.5| null|       Q|
|         26|       1|     3|Asplund, Mrs. Car...|female|38.0|    1|    5|          347077|31.3875| null|       S|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|         27|       0|     3|Emir, Mr. Farred ...|  male|null|    0|    0|            2631|  7.225| null|       C|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [11]:
# Pclass 오름차순 , Age 내림차순
titanic_sdf.orderBy(
    [F.col("Pclass"),F.col("Age")],
    ascending=[False, True]
).show()

+-----------+--------+------+--------------------+------+----+-----+-----+---------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|         Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+---------------+-------+-----+--------+
|         78|       0|     3|Moutal, Mr. Raham...|  male|null|    0|    0|         374746|   8.05| null|       S|
|        177|       0|     3|Lefebre, Master. ...|  male|null|    3|    1|           4133|25.4667| null|       S|
|         83|       1|     3|McDermott, Miss. ...|female|null|    0|    0|         330932| 7.7875| null|       Q|
|         77|       0|     3|   Staneff, Mr. Ivan|  male|null|    0|    0|         349208| 7.8958| null|       S|
|         88|       0|     3|Slocovski, Mr. Se...|  male|null|    0|    0|SOTON/OQ 392086|   8.05| null|       S|
|         37|       1|     3|    Mamee, Mr. Hanna|  male|null|    0|    0|           267

In [13]:
# orderBy 와 동일한 메소드인 sort
titanic_sdf.sort(
    F.col("Pclass").asc(),
    F.col("Age").desc()
).show()

+-----------+--------+------+--------------------+------+----+-----+-----+---------+--------+-----------+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|   Ticket|    Fare|      Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+---------+--------+-----------+--------+
|        631|       1|     1|Barkworth, Mr. Al...|  male|80.0|    0|    0|    27042|    30.0|        A23|       S|
|        494|       0|     1|Artagaveytia, Mr....|  male|71.0|    0|    0| PC 17609| 49.5042|       null|       C|
|         97|       0|     1|Goldschmidt, Mr. ...|  male|71.0|    0|    0| PC 17754| 34.6542|         A5|       C|
|        746|       0|     1|Crosby, Capt. Edw...|  male|70.0|    1|    1|WE/P 5735|    71.0|        B22|       S|
|         55|       0|     1|Ostby, Mr. Engelh...|  male|65.0|    0|    1|   113509| 61.9792|        B30|       C|
|        457|       0|     1|Millet, Mr. Franc...|  male|65.0|    0|    0|    13

## 연습 문제
다음 쿼리를 스파크 데이터 프레임으로 표현하세요..
1. ```SQL
    select Pclass, Name
    from ( select *
           from titanic_sdf
           order by Pclass asc, Name desc)
```

2. ```SQL
select Pclass, Name
from titanic_sdf
order by Pclass asc, Name desc
```

In [15]:
# 1
titanic_sdf.sort(
    F.col("Pclass").asc(),
    F.col("Name").desc()
).select(
    F.col("Pclass"),
    F.col("Name")
).show()

+------+--------------------+
|Pclass|                Name|
+------+--------------------+
|     1|Young, Miss. Mari...|
|     1|  Wright, Mr. George|
|     1|   Woolner, Mr. Hugh|
|     1|Williams-Lambert,...|
|     1|Williams, Mr. Cha...|
|     1|Widener, Mr. Harr...|
|     1|Wick, Mrs. George...|
|     1|Wick, Miss. Mary ...|
|     1|White, Mr. Richar...|
|     1|White, Mr. Perciv...|
|     1|     Weir, Col. John|
|     1|Warren, Mrs. Fran...|
|     1|    Ward, Miss. Anna|
|     1|Walker, Mr. Willi...|
|     1|Van der hoef, Mr....|
|     1|Uruchurtu, Don. M...|
|     1|Thorne, Mrs. Gert...|
|     1|Thayer, Mrs. John...|
|     1|Thayer, Mr. John ...|
|     1|Thayer, Mr. John ...|
+------+--------------------+
only showing top 20 rows



In [18]:
# 2
titanic_sdf.select(
    F.col("Pclass"), F.col("Name")
).orderBy(
    [F.col("Pclass"), F.col("Name")],
    ascending=[True, False]
).show()

+------+--------------------+
|Pclass|                Name|
+------+--------------------+
|     1|Young, Miss. Mari...|
|     1|  Wright, Mr. George|
|     1|   Woolner, Mr. Hugh|
|     1|Williams-Lambert,...|
|     1|Williams, Mr. Cha...|
|     1|Widener, Mr. Harr...|
|     1|Wick, Mrs. George...|
|     1|Wick, Miss. Mary ...|
|     1|White, Mr. Richar...|
|     1|White, Mr. Perciv...|
|     1|     Weir, Col. John|
|     1|Warren, Mrs. Fran...|
|     1|    Ward, Miss. Anna|
|     1|Walker, Mr. Willi...|
|     1|Van der hoef, Mr....|
|     1|Uruchurtu, Don. M...|
|     1|Thorne, Mrs. Gert...|
|     1|Thayer, Mrs. John...|
|     1|Thayer, Mr. John ...|
|     1|Thayer, Mr. John ...|
+------+--------------------+
only showing top 20 rows



# Aggregation
- Spark DataFrame은 DataFrame 객체에서 집계 메소드가 많이 없다.
- 집계에 관련된 메소드는 거의 대부분 `pyspark.sql.functions` 패키지에 있는 함수를 활용

In [19]:
# spark에는 집계 함수가 없음
titanic_sdf.select(
    F.col("Age")
).mean()

AttributeError: 'DataFrame' object has no attribute 'mean'

In [20]:
# Spark DataFrame 으로 집계하기
import pyspark.sql.functions as F

titanic_sdf_agg = titanic_sdf.select(
    F.max(F.col("Age")),
    F.min(F.col("Age")),
    F.sum(F.col("Age"))
)

titanic_sdf_agg.show()

+--------+--------+--------+
|max(Age)|min(Age)|sum(Age)|
+--------+--------+--------+
|    80.0|    0.42|21205.17|
+--------+--------+--------+



In [21]:
titanic_sdf.groupby(
    F.col("Pclass")
).count().show()

+------+-----+
|Pclass|count|
+------+-----+
|     1|  216|
|     3|  491|
|     2|  184|
+------+-----+



In [22]:
# count 가 아닌 다른 집계 함수를 groupby를 이용해 특정 칼럼에 적용하면 판다스 데이터 프레임과 유사하게 나온다
titanic_sdf.groupby("Pclass").max().show()

+------+----------------+-------------+-----------+--------+----------+----------+---------+
|Pclass|max(PassengerId)|max(Survived)|max(Pclass)|max(Age)|max(SibSp)|max(Parch)|max(Fare)|
+------+----------------+-------------+-----------+--------+----------+----------+---------+
|     1|             890|            1|          1|    80.0|         3|         4| 512.3292|
|     3|             891|            1|          3|    74.0|         8|         6|    69.55|
|     2|             887|            1|          2|    70.0|         3|         3|     73.5|
+------+----------------+-------------+-----------+--------+----------+----------+---------+



In [26]:
# max 와 같은 집계 함수 내에서는 컬럼 속성을 사용 할 수 없음(예:max(F.col("Age")))
titanic_sdf.groupby("Pclass").max("Age").show()

+------+--------+
|Pclass|max(Age)|
+------+--------+
|     1|    80.0|
|     3|    74.0|
|     2|    70.0|
+------+--------+



In [27]:
# 두 개 이상의 집계 기준 마련하기
titanic_sdf.groupby(
    [F.col("Pclass"), F.col("Embarked")]
).max("Age").show()

+------+--------+--------+
|Pclass|Embarked|max(Age)|
+------+--------+--------+
|     3|       C|    45.5|
|     2|       C|    36.0|
|     1|    null|    62.0|
|     1|       Q|    44.0|
|     3|       Q|    70.5|
|     2|       Q|    57.0|
|     1|       C|    71.0|
|     1|       S|    80.0|
|     3|       S|    74.0|
|     2|       S|    70.0|
+------+--------+--------+



In [28]:
# agg 함수 사용하기 
# function 패키지 안에는 또 컬럼 속성 넣을 수 있다
titanic_sdf.groupby("Pclass").agg(
    F.max(F.col("Age")).alias("max_age"),
    F.min("Age").alias("min_age"),
    F.sum(F.col("Age")).alias("sum_age"),
    F.avg("Age").alias("avg_Age")
).show()

+------+-------+-------+-------+------------------+
|Pclass|max_age|min_age|sum_age|           avg_Age|
+------+-------+-------+-------+------------------+
|     1|   80.0|   0.92|7111.42|38.233440860215055|
|     3|   74.0|   0.42|8924.92| 25.14061971830986|
|     2|   70.0|   0.67|5168.83| 29.87763005780347|
+------+-------+-------+-------+------------------+



In [29]:
spark.stop()